## Dependencies

In [ ]:
!pip install kglab
!pip install sentence-transformers
!pip install transformers


## Imports

In [ ]:
import pandas as pd
import numpy as np
import re

import pandas as pd
import kglab as kglab
import rdflib
from rdflib import Graph, URIRef, Namespace
from rdflib import Literal, XSD

from sentence_transformers import SentenceTransformer
from transformers import pipeline

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df_dataframe=pd.read_csv('dataframes_with_labels.csv')
df_dataframe.head()

,DataFrame has,Syntax,Description:,Label,Type
0,Constructor,"DataFrame([data, index, columns, dtype, copy])","Two-dimensional, size-mutable, potentially het...",constructor,Function
1,Attributes and underlying data,DataFrame.index,The index (row labels) of the DataFrame.,index information,Attribute
2,Attributes and underlying data,DataFrame.columns,The column labels of the DataFrame.,column label,Attribute
3,Attributes and underlying data,DataFrame.dtypes,Return the dtypes in the DataFrame.,find datatypes,Attribute
4,Attributes and underlying data,"DataFrame.info([verbose, buf, max_cols, ...])",Print a concise summary of a DataFrame.,data summary,Function


In [ ]:
df_dataframe.shape

(227, 5)

In [ ]:
df_io=pd.read_csv('io_With_labels.csv')
df_io.head()

,PandasHas,Syntax,Description:,Label,Type
0,Pickling,"read_pickle(filepath_or_buffer[, ...])",Load pickled pandas object (or any object) fro...,read pickle,Function
1,Pickling,"DataFrame.to_pickle(path[, compression, ...])",Pickle (serialize) object to file.,write pickle,Function
2,Flat file,"read_table(filepath_or_buffer[, sep, ...])",Read general delimited file into DataFrame.,read table,Function
3,Flat file,"read_csv(filepath_or_buffer[, sep, ...])",Read a comma-separated values (csv) file into ...,read csv,Function
4,Flat file,"DataFrame.to_csv([path_or_buf, sep, na_rep, ...])",Write object to a comma-separated values (csv)...,write csv,Function


In [ ]:
df_io.shape

(53, 5)

___________________________________________________________________________________________________________________________________________________________________________________________________________

_________________________________________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
labels=set(df_dataframe.Label)
print(len(labels))
labels=labels.union(set(df_io.Label))
print(len(labels))

labels=labels.union(set(df_series.Labels))
len(labels)

211
247


517

In [ ]:
labels=list(labels)

In [ ]:
labels=pd.Series(labels)
labels.head()

0                            NaN
1    convert to another timezone
2     smallest group of elements
3             subtract to series
4                   series flags
dtype: object

In [ ]:
labels.dropna(inplace=True)
print(labels.isna().sum())
print(labels.shape)

0
(516,)


In [ ]:
labels.to_csv('labels.csv',index=False)

In [ ]:
df_dataframe['parameters']=None
for index,row in df_dataframe.iterrows():
    
    if row['Type']=='Function':
        pram=[]
        description_sentence=row['Syntax'].split('(')
        description_sentence=description_sentence[1]
        description_sentence=description_sentence.split(')')
        description_sentence=description_sentence[0]
        description_sentence=description_sentence.replace('[','')


        description_sentence=description_sentence.replace(']','')
        description_sentence=description_sentence.split(',')
        for j in description_sentence:
            pram.append(j.strip())
        df_dataframe['parameters'][index]=pram


df_dataframe['cut_syntax']=df_dataframe['Syntax']
for index,row in df_dataframe.iterrows():
    if row['Type']=='Function':
        syn=(row['Syntax'].split('('))[0]
        syn=syn+'()'
        row['cut_syntax']=syn


df_dataframe.head(7)

,DataFrame has,Syntax,Description:,Label,Type,parameters,cut_syntax
0,Constructor,"DataFrame([data, index, columns, dtype, copy])","Two-dimensional, size-mutable, potentially het...",constructor,Function,"[data, index, columns, dtype, copy]",DataFrame()
1,Attributes and underlying data,DataFrame.index,The index (row labels) of the DataFrame.,index information,Attribute,None,DataFrame.index
2,Attributes and underlying data,DataFrame.columns,The column labels of the DataFrame.,column label,Attribute,None,DataFrame.columns
3,Attributes and underlying data,DataFrame.dtypes,Return the dtypes in the DataFrame.,find datatypes,Attribute,None,DataFrame.dtypes
4,Attributes and underlying data,"DataFrame.info([verbose, buf, max_cols, ...])",Print a concise summary of a DataFrame.,data summary,Function,"[verbose, buf, max_cols, ...]",DataFrame.info()
5,Attributes and underlying data,"DataFrame.select_dtypes([include, exclude])",Return a subset of the DataFrame's columns bas...,select datatypes,Function,"[include, exclude]",DataFrame.select_dtypes()
6,Attributes and underlying data,DataFrame.values,Return a Numpy representation of the DataFrame.,dataframe to numpy,Attribute,None,DataFrame.values


In [ ]:
df_dataframe.to_csv('dataframes_with_labels.csv',index=False)

In [ ]:
df_io
df_io['parameters']=None
for index,row in df_io.iterrows():
    
    if row['Type']=='Function':
        pram=[]
        description_sentence=row['Syntax'].split('(')
        description_sentence=description_sentence[1]
        description_sentence=description_sentence.split(')')
        description_sentence=description_sentence[0]
        description_sentence=description_sentence.replace('[','')


        description_sentence=description_sentence.replace(']','')
        description_sentence=description_sentence.split(',')
        for j in description_sentence:
            pram.append(j.strip())
        df_io['parameters'][index]=pram


df_io['cut_syntax']=df_io['Syntax']
for index,row in df_io.iterrows():
    if row['Type']=='Function':
        syn=(row['Syntax'].split('('))[0]
        syn=syn+'()'
        row['cut_syntax']=syn


df_io.head(7)

,PandasHas,Syntax,Description:,Label,Type,parameters,cut_syntax
0,Pickling,"read_pickle(filepath_or_buffer[, ...])",Load pickled pandas object (or any object) fro...,read pickle,Function,"[filepath_or_buffer, ...]",read_pickle()
1,Pickling,"DataFrame.to_pickle(path[, compression, ...])",Pickle (serialize) object to file.,write pickle,Function,"[path, compression, ...]",DataFrame.to_pickle()
2,Flat file,"read_table(filepath_or_buffer[, sep, ...])",Read general delimited file into DataFrame.,read table,Function,"[filepath_or_buffer, sep, ...]",read_table()
3,Flat file,"read_csv(filepath_or_buffer[, sep, ...])",Read a comma-separated values (csv) file into ...,read csv,Function,"[filepath_or_buffer, sep, ...]",read_csv()
4,Flat file,"DataFrame.to_csv([path_or_buf, sep, na_rep, ...])",Write object to a comma-separated values (csv)...,write csv,Function,"[path_or_buf, sep, na_rep, ...]",DataFrame.to_csv()
5,Flat file,"read_fwf(filepath_or_buffer[, colspecs, ...])",Read a table of fixed-width formatted lines in...,read fixed width table,Function,"[filepath_or_buffer, colspecs, ...]",read_fwf()
6,Clipboard,read_clipboard([sep]),Read text from clipboard and pass to read_csv.,read clipboard,Function,[sep],read_clipboard()


In [ ]:
df_io.to_csv('io_With_labels.csv',index=False)

In [ ]:
df_series['Type']=np.nan
for index,row in df_series.iterrows():
    if ')' in df_series['Syntax'][index] and '(' in df_series['Syntax'][index]:
        df_series['Type'][index]='Function'
    else:
        df_series['Type'][index]='Attribute'

df_series.head(3)

,Series has,Syntax,Description:,Label,parameters,Type,cut_syntax
0,Constructor,"Series([data, index, dtype, name, copy, ...])",One-dimensional ndarray with axis labels (incl...,constructor,"['data', 'index', 'dtype', 'name', 'copy', '...']",Function,Series()
1,Attributes,Series.index,The index (axis labels) of the Series.,index information,NaN,Attribute,Series.index
2,Attributes,Series.array,The ExtensionArray of the data backing this Se...,convert to array,NaN,Attribute,Series.array


In [ ]:
df_series


df_series['parameters']=None
for index,row in df_series.iterrows():
    
    if row['Type']=='Function':
        pram=[]
        description_sentence=row['Syntax'].split('(')
        description_sentence=description_sentence[1]
        description_sentence=description_sentence.split(')')
        description_sentence=description_sentence[0]
        description_sentence=description_sentence.replace('[','')


        description_sentence=description_sentence.replace(']','')
        description_sentence=description_sentence.split(',')
        for j in description_sentence:
            pram.append(j.strip())
        df_series['parameters'][index]=pram


df_series['cut_syntax']=df_series['Syntax']
for index,row in df_series.iterrows():
    if row['Type']=='Function':
        syn=(row['Syntax'].split('('))[0]
        syn=syn+'()'
        row['cut_syntax']=syn


df_series.head(7)

,Series has,Syntax,Description:,Label,parameters,Type,cut_syntax
0,Constructor,"Series([data, index, dtype, name, copy, ...])",One-dimensional ndarray with axis labels (incl...,constructor,"[data, index, dtype, name, copy, ...]",Function,Series()
1,Attributes,Series.index,The index (axis labels) of the Series.,index information,None,Attribute,Series.index
2,Attributes,Series.array,The ExtensionArray of the data backing this Se...,convert to array,None,Attribute,Series.array
3,Attributes,Series.values,Return Series as ndarray or ndarray-like depen...,find values,None,Attribute,Series.values
4,Attributes,Series.dtype,Return the dtype object of the underlying data.,find datatypes,None,Attribute,Series.dtype
5,Attributes,Series.shape,Return a tuple of the shape of the underlying ...,find rows and columns,None,Attribute,Series.shape
6,Attributes,Series.nbytes,Return the number of bytes in the underlying d...,find bytes,None,Attribute,Series.nbytes


In [ ]:
df_series.to_csv('series_with_labels.csv',index=False)

__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

# **Build Knowledge Graph**

### Read Data

In [ ]:
df_dataframe=pd.read_csv('/content/drive/MyDrive/DVA Knowledge Graph/dataframes_with_labels.csv')
df_dataframe.head()

,DataFrame has,Syntax,Description:,Label,Type,parameters,cut_syntax
0,Constructor,"DataFrame([data, index, columns, dtype, copy])","Two-dimensional, size-mutable, potentially het...",constructor,Function,"['data', 'index', 'columns', 'dtype', 'copy']",DataFrame()
1,Attributes and underlying data,DataFrame.index,The index (row labels) of the DataFrame.,index information,Attribute,NaN,DataFrame.index
2,Attributes and underlying data,DataFrame.columns,The column labels of the DataFrame.,column label,Attribute,NaN,DataFrame.columns
3,Attributes and underlying data,DataFrame.dtypes,Return the dtypes in the DataFrame.,find datatypes,Attribute,NaN,DataFrame.dtypes
4,Attributes and underlying data,"DataFrame.info([verbose, buf, max_cols, ...])",Print a concise summary of a DataFrame.,data summary,Function,"['verbose', 'buf', 'max_cols', '...']",DataFrame.info()


In [ ]:
df_io=pd.read_csv('/content/drive/MyDrive/DVA Knowledge Graph/io_with_labels.csv')
df_io.head()

,PandasHas,Syntax,Description:,Label,Type,parameters,cut_syntax
0,Pickling,"read_pickle(filepath_or_buffer[, ...])",Load pickled pandas object (or any object) fro...,read pickle,Function,"['filepath_or_buffer', '...']",read_pickle()
1,Pickling,"DataFrame.to_pickle(path[, compression, ...])",Pickle (serialize) object to file.,write pickle,Function,"['path', 'compression', '...']",DataFrame.to_pickle()
2,Flat file,"read_table(filepath_or_buffer[, sep, ...])",Read general delimited file into DataFrame.,read table,Function,"['filepath_or_buffer', 'sep', '...']",read_table()
3,Flat file,"read_csv(filepath_or_buffer[, sep, ...])",Read a comma-separated values (csv) file into ...,read csv,Function,"['filepath_or_buffer', 'sep', '...']",read_csv()
4,Flat file,"DataFrame.to_csv([path_or_buf, sep, na_rep, ...])",Write object to a comma-separated values (csv)...,write csv,Function,"['path_or_buf', 'sep', 'na_rep', '...']",DataFrame.to_csv()


In [ ]:
df_series=pd.read_csv('/content/drive/MyDrive/DVA Knowledge Graph/series_with_labels.csv')
df_series.columns=['Series has', 'Syntax', 'Description:', 'Label',  'parameters','Type',
       'cut_syntax']
df_series.head()

,Series has,Syntax,Description:,Label,parameters,Type,cut_syntax
0,Constructor,"Series([data, index, dtype, name, copy, ...])",One-dimensional ndarray with axis labels (incl...,constructor,"['data', 'index', 'dtype', 'name', 'copy', '...']",Function,Series()
1,Attributes,Series.index,The index (axis labels) of the Series.,index information,NaN,Attribute,Series.index
2,Attributes,Series.array,The ExtensionArray of the data backing this Se...,convert to array,NaN,Attribute,Series.array
3,Attributes,Series.values,Return Series as ndarray or ndarray-like depen...,find values,NaN,Attribute,Series.values
4,Attributes,Series.dtype,Return the dtype object of the underlying data.,find datatypes,NaN,Attribute,Series.dtype


In [ ]:
df_series.shape

(343, 7)

---

We will be building two knowledge graphs, One main knowledge graph to store libaries and functions. The other one is to store input and output parameters for each function

# **Parameter Knowledge Graph**

In [ ]:

#Defining Namspaces for Knowledge Graph
namespaces_pram = {
    "wtm": "http://example.org/python/",
    "syn": "http://example.org/syntax/",
    "pram": "http://example.org/parameter/",
    }

#Setting the Parameters
kg_pram = kglab.KnowledgeGraph(
    name = "A Python Libaries Data",
    namespaces = namespaces_pram,
    )


#Setting the main/master node
mainnode=rdflib.URIRef(kg_pram.get_ns('wtm')+'Function')

#Iterate over dataframe
for index,row in df_dataframe.iterrows():
  if row['Type']=='Function':
    syntax=row['cut_syntax']
    syntax.replace('(','')
    syntax.replace(')','')
    syntax=rdflib.URIRef(kg_pram.get_ns('syn')+str(syntax))
    #Adding Syntax node to knowledge graph
    kg_pram.add(mainnode,kg_pram.get_ns("wtm").hasSyntax,syntax)
    prm_list=row['parameters']
    for j in prm_list:
      if j!='...':
        #Adding parameter for that specific function and syntax.
        prm_node=rdflib.URIRef(kg_pram.get_ns('pram')+j.strip())
        kg_pram.add(syntax,kg_pram.get_ns('wtm').hasParameter,prm_node)
  else:
    pass

for index,row in df_series.iterrows():
  if row['Type']=='Function':
    syntax=row['cut_syntax']
    syntax.replace('(','')
    syntax.replace(')','')
    syntax=rdflib.URIRef(kg_pram.get_ns('syn')+str(syntax))
    kg_pram.add(mainnode,kg_pram.get_ns("wtm").hasSyntax,syntax)
    prm_list=row['parameters']
    for j in prm_list:
      if j!='...':
        prm_node=rdflib.URIRef(kg_pram.get_ns('pram')+j.strip())
        kg_pram.add(syntax,kg_pram.get_ns('wtm').hasParameter,prm_node)
  else:
    pass

for index,row in df_io.iterrows():
  if row['Type']=='Function':
    syntax=row['cut_syntax']
    syntax.replace('(','')
    syntax.replace(')','')
    syntax=rdflib.URIRef(kg_pram.get_ns('syn')+str(syntax))
    kg_pram.add(mainnode,kg_pram.get_ns("wtm").hasSyntax,syntax)
    prm_list=row['parameters']
    for j in prm_list:
      if j!='...':
        prm_node=rdflib.URIRef(kg_pram.get_ns('pram')+j.strip())
        kg_pram.add(syntax,kg_pram.get_ns('wtm').hasParameter,prm_node)
  else:
    pass




In [ ]:
VIS_STYLE = {
    "wtm": {
        "color": "black",
        "size": 250,
    },
    "pram":{
        "color": "purple",
        "size": 200,
    },
    "syn":{
        "color": "green",
        "size": 25,
    }
}

subgraph = kglab.SubgraphTensor(kg_pram)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.force_atlas_2based()
pyvis_graph.show("parameter_graph_df.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# **Knowledge Graphs for Querying**

### **DataFrame**

In [ ]:
namespaces = {
    "wtm": "http://example.org/python/",
    "lib": "http://example.org/library/",
    "df": "http://example.org/dataframe/",
    "io": "http://example.org/inputoutput/",
    "ser": "http://example.org/series/",
    "syn": "http://example.org/syntax/",
    "des": "http://example.org/description/",
    "typ": "http://example.org/type/",
    "pram": "http://example.org/parameter/",
    "lab": "http://example.org/label/"
    }

kg = kglab.KnowledgeGraph(
    name = "A Python Libaries Data",
    namespaces = namespaces,
    )


main_node=Literal('python')

library_name=rdflib.URIRef(kg.get_ns('lib')+'Pandas')
kg.add(library_name, kg.get_ns("lib").LibraryName, kg.get_ns("wtm").Python)
dataframenode=rdflib.URIRef(kg.get_ns("df")+'Dataframe')
kg.add(library_name,kg.get_ns("wtm").hasClass,dataframenode)

################################################################################################################################################################

for index,row in df_dataframe.iterrows():
    
    syn=rdflib.URIRef(kg.get_ns("syn")+str(row['cut_syntax']))

    if row['Type']=='Attribute':
        kg.add(dataframenode,kg.get_ns("wtm").hasAttribute,syn)
    else:
        kg.add(dataframenode,kg.get_ns("wtm").hasFunction,syn)

    x=str(row['Description:']).split(' ')
    des_str=x[0]
    for i in range(1,len(x)):
        des_str=des_str+'_'+x[i]
    des_str=des_str.replace('"','')
    des_str=des_str.replace('<','')
    des_str=des_str.replace('>','')
    des=rdflib.URIRef(kg.get_ns("des")+des_str)
    kg.add(syn,kg.get_ns("wtm").hasDescription,des)
    
    x=str(row['Label']).split(' ')
    des_str=x[0]
    for i in range(1,len(x)):
        des_str=des_str+'_'+x[i]
    lab=rdflib.URIRef(kg.get_ns("lab")+des_str)
    kg.add(syn,kg.get_ns("wtm").hasLabel,lab)



__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

### **IO Methods**

In [ ]:
ionode=rdflib.URIRef(kg.get_ns("io")+'Input_Output')
kg.add(library_name,kg.get_ns("wtm").hasClass,ionode)


################################################################################################################################################################

df_io

for index,row in df_io.iterrows():
    
    syn=rdflib.URIRef(kg.get_ns("syn")+str(row['cut_syntax']))

    kg.add(ionode,kg.get_ns("wtm").hasFunction,syn)

    x=str(row['Description:']).split(' ')
    des_str=x[0]
    for i in range(1,len(x)):
        des_str=des_str+'_'+x[i]
    des_str=des_str.replace('"','')
    des_str=des_str.replace('<','')
    des_str=des_str.replace('>','')
    des=rdflib.URIRef(kg.get_ns("des")+des_str)
    kg.add(syn,kg.get_ns("wtm").hasDescription,des)
    
    x=str(row['Label']).split(' ')
    des_str=x[0]
    for i in range(1,len(x)):
        des_str=des_str+'_'+x[i]
    lab=rdflib.URIRef(kg.get_ns("lab")+des_str)
    kg.add(syn,kg.get_ns("wtm").hasLabel,lab)

In [ ]:
VIS_STYLE = {
    "wtm": {
        "color": "black",
        "size": 250,
    },
    "lib":{
        "color": "green",
        "size": 250,
    },
    "df":{
        "color": "red",
        "size": 200,
    },
    "io":{
        "color": "purple",
        "size": 200,
    },
    "syn":{
        "color": "blue",
        "size": 25,
    }
}

subgraph = kglab.SubgraphTensor(kg)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.force_atlas_2based()
pyvis_graph.show("sample_kg_pandas_df_io.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### **Series**

In [ ]:
seriesnode=rdflib.URIRef(kg.get_ns("ser")+'Series')
kg.add(library_name,kg.get_ns("wtm").hasClass,seriesnode)


################################################################################################################################################################
df_series


for index,row in df_series.iterrows():
    
    syn=rdflib.URIRef(kg.get_ns("syn")+str(row['cut_syntax']))

    kg.add(seriesnode,kg.get_ns("wtm").hasFunction,syn)

    x=str(row['Description:']).split(' ')
    des_str=x[0]
    for i in range(1,len(x)):
        des_str=des_str+'_'+x[i]
    des_str=des_str.replace('"','')
    des_str=des_str.replace('<','')
    des_str=des_str.replace('>','')
    des_str=des_str.replace('.','')
    des_str=des_str.replace('{','')
    des_str=des_str.replace('}','')
    des_str=des_str.replace('-','')
    des=rdflib.URIRef(kg.get_ns("des")+des_str)
    kg.add(syn,kg.get_ns("wtm").hasDescription,des)
    
    x=str(row['Label']).split(' ')
    des_str=x[0]
    for i in range(1,len(x)):
        des_str=des_str+'_'+x[i]
    lab=rdflib.URIRef(kg.get_ns("lab")+des_str)
    kg.add(syn,kg.get_ns("wtm").hasLabel,lab)

In [ ]:
VIS_STYLE = {
    "wtm": {
        "color": "black",
        "size": 250,
    },
    "lib":{
        "color": "green",
        "size": 250,
    },
    "df":{
        "color": "red",
        "size": 200,
    },
    "io":{
        "color": "purple",
        "size": 200,
    },
    "ser":{
        "color": "yellow",
        "size": 350,
    },
    "syn":{
        "color": "blue",
        "size": 25,
    }
}

subgraph = kglab.SubgraphTensor(kg)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.force_atlas_2based()
pyvis_graph.show("sample_kg_pandas_df_io_series.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# **SPARQL Query From Knowledge Graph**

In [ ]:
sparql1 = """
SELECT *
  WHERE {
      ?Library lib:LibraryName  wtm:Python .
      ?Library wtm:hasClass ?Class.
      ?Class  wtm:hasAttribute ?syn.
      ?syn wtm:hasDescription ?desc.
      ?syn wtm:hasLabel ?label
      }
  """

dfquery1 = kg.query_as_df(sparql1)
dfquery1

,Library,Class,desc,label,syn
0,lib:Pandas,df:Dataframe,des:The_index_(row_labels)_of_the_DataFrame.,lab:index_information,syn:DataFrame.index
1,lib:Pandas,df:Dataframe,des:The_column_labels_of_the_DataFrame.,lab:column_label,syn:DataFrame.columns
2,lib:Pandas,df:Dataframe,des:Return_the_dtypes_in_the_DataFrame.,lab:find_datatypes,syn:DataFrame.dtypes
3,lib:Pandas,df:Dataframe,des:Return_a_Numpy_representation_of_the_DataF...,lab:dataframe_to_numpy,syn:DataFrame.values
4,lib:Pandas,df:Dataframe,des:Return_a_list_representing_the_axes_of_the...,lab:axes_labels,syn:DataFrame.axes
5,lib:Pandas,df:Dataframe,<http://example.org/description/Return_an_int_...,lab:dimention,syn:DataFrame.ndim
6,lib:Pandas,df:Dataframe,des:Return_an_int_representing_the_number_of_e...,lab:size_of_data,syn:DataFrame.size
7,lib:Pandas,df:Dataframe,des:Return_a_tuple_representing_the_dimensiona...,lab:shape_of_data,syn:DataFrame.shape
8,lib:Pandas,df:Dataframe,<http://example.org/description/Indicator_whet...,lab:empty_or_not,syn:DataFrame.empty
9,lib:Pandas,df:Dataframe,<http://example.org/description/Access_a_singl...,lab:locate_a_cell,syn:DataFrame.at


In [ ]:
lab='lab:column_label'

In [ ]:
sparql1 = """
SELECT *
  WHERE {
      ?Library lib:LibraryName  wtm:Python .
      ?Library wtm:hasClass ?Class.
      ?Class  wtm:hasFunction ?syn.
      ?syn wtm:hasDescription ?desc.
      ?syn wtm:hasLabel ?label
      }
  """

dfquery1 = kg.query_as_df(sparql1)
dfquery1

,Library,Class,desc,label,syn
0,lib:Pandas,df:Dataframe,<http://example.org/description/Two-dimensiona...,lab:constructor,syn:DataFrame()
1,lib:Pandas,df:Dataframe,des:Print_a_concise_summary_of_a_DataFrame.,lab:data_summary,syn:DataFrame.info()
2,lib:Pandas,df:Dataframe,<http://example.org/description/Return_a_subse...,lab:select_datatypes,syn:DataFrame.select_dtypes()
3,lib:Pandas,df:Dataframe,des:Return_the_memory_usage_of_each_column_in_...,lab:memory_usage,syn:DataFrame.memory_usage()
4,lib:Pandas,df:Dataframe,des:Return_a_new_object_with_updated_flags.,lab:allow_duplicates_or_not,syn:DataFrame.set_flags()
...,...,...,...,...,...
626,lib:Pandas,ser:Series,des:Convert_the_object_to_a_JSON_string,lab:conver_to_JSON,syn:Series.to_json()
627,lib:Pandas,ser:Series,des:Render_a_string_representation_of_the_Series,lab:convert_to_string,syn:Series.to_string()
628,lib:Pandas,ser:Series,des:Copy_object_to_the_system_clipboard,lab:copy_to_clipboard,syn:Series.to_clipboard()
629,lib:Pandas,ser:Series,<http://example.org/description/Render_object_...,lab:conver_to_latex,syn:Series.to_latex()




---



---



---



# **Natural Language Query**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DVA Knowledge Graph/labels.csv')
df.head()

,0
0,convert to another timezone
1,smallest group of elements
2,subtract to series
3,series flags
4,lenght of strings


In [ ]:
labels = list(df['0'])
types = list()
for i in range(0,len(labels)):
  labels[i] = str(labels[i])
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(labels)
sen_embeddings.shape
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
sentence='I want to read data from text file seprated by comma'
test = model.encode(sentence)
test = test.reshape(1,-1)

from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
array = cosine_similarity(
    test,
    sen_embeddings[0:])

n = 10
idx = (-array[0]).argsort()[:n]

predictions=[]
for i in idx:
  predictions.append(labels[i].strip())

zero_shot_pred=classifier(sentence, predictions,multi_label=True)

df_queries=[]

for i in range(3):

  desired_label=zero_shot_pred['labels'][i]
  desirec_label=desired_label.strip().replace(' ','_')

  sparql1 = """
  SELECT *
    WHERE {
        ?Library lib:LibraryName  wtm:Python .
        ?Library wtm:hasClass ?Class.
        ?Class  wtm:hasFunction ?Syntax.
        ?Syntax wtm:hasDescription ?desc.
        ?Syntax wtm:hasLabel %s
        }
    """
    
  sparql1=sparql1 % ('lab:'+desirec_label)

  dfquery = kg.query_as_df(sparql1)

  if dfquery.empty:
    sparql1 = """
    SELECT *
      WHERE {
          ?Library lib:LibraryName  wtm:Python .
          ?Library wtm:hasClass ?Class.
          ?Class  wtm:hasAttribute ?Syntax.
          ?Syntax wtm:hasDescription ?desc.
          ?Syntax wtm:hasLabel %s
          }
      """
      
    sparql1=sparql1 % ('lab:'+desirec_label)

    dfquery = kg.query_as_df(sparql1)

  df_queries.append(dfquery)

df_query_result=pd.concat(df_queries,axis=0)
df_query_result

,Library,desc,Syntax,Class
0,lib:Pandas,des:Read_a_comma-separated_values_(csv)_file_i...,syn:read_csv(),io:Input_Output
0,lib:Pandas,des:Write_the_contained_data_to_an_HDF5_file_u...,syn:DataFrame.to_hdf(),df:Dataframe
0,lib:Pandas,des:Evaluate_a_string_describing_operations_on...,syn:DataFrame.eval(),df:Dataframe


Let's Put in into a Function

In [ ]:
def reply(sentence,embedding_model,classifier,kg):
    sentence='I want to read data from text file seprated by comma'
    test = model.encode(sentence)
    test = test.reshape(1,-1)

    from sklearn.metrics.pairwise import cosine_similarity
    #let's calculate cosine similarity for sentence 0:
    array = cosine_similarity(
        test,
        sen_embeddings[0:])

    n = 10
    idx = (-array[0]).argsort()[:n]

    predictions=[]
    for i in idx:
    predictions.append(labels[i].strip())

    zero_shot_pred=classifier(sentence, predictions,multi_label=True)

    df_queries=[]

    for i in range(3):

    desired_label=zero_shot_pred['labels'][i]
    desirec_label=desired_label.strip().replace(' ','_')

    sparql1 = """
    SELECT *
        WHERE {
            ?Library lib:LibraryName  wtm:Python .
            ?Library wtm:hasClass ?Class.
            ?Class  wtm:hasFunction ?Syntax.
            ?Syntax wtm:hasDescription ?desc.
            ?Syntax wtm:hasLabel %s
            }
        """
        
    sparql1=sparql1 % ('lab:'+desirec_label)

    dfquery = kg.query_as_df(sparql1)

    if dfquery.empty:
        sparql1 = """
        SELECT *
        WHERE {
            ?Library lib:LibraryName  wtm:Python .
            ?Library wtm:hasClass ?Class.
            ?Class  wtm:hasAttribute ?Syntax.
            ?Syntax wtm:hasDescription ?desc.
            ?Syntax wtm:hasLabel %s
            }
        """
        
        sparql1=sparql1 % ('lab:'+desirec_label)

        dfquery = kg.query_as_df(sparql1)

    df_queries.append(dfquery)

    df_query_result=pd.concat(df_queries,axis=0)
    return df_query_result
    

In [ ]:
reply('I want to read data from text file seprated by comma',embedding_model,classifier,kg)




---



---





---



---



# **Let's calculate Accuracy and other Statistics**

In [ ]:
df_questions=pd.read_csv('/content/drive/MyDrive/DVA Knowledge Graph/Question.csv')
df_questions.columns=['Questions','true_answer_in_top']
df_questions.head(2)

,Questions,true_answer_in_top
0,get the shape of dataframe,3
1,how to insert a column,1


In [ ]:
df_questions['true_answer_in_top'].mean()

1.391304347826087

**Thus on average, the true result is returned in 1st Query**

In [ ]:
counts=df_questions['true_answer_in_top'].value_counts()
counts

1    14
3     3
0     3
2     2
5     1
Name: true_answer_in_top, dtype: int64

In [ ]:
accuracy=(1-counts[0]/df_questions.shape[0])*100
accuracy

86.95652173913044

**Thus  Accuracy is 87%** 



---



---



---
